In [ ]:
!pip install xgboost

In [ ]:
import spectral
import matplotlib.pyplot as plt
import numpy as np
import spectral.io.aviris as aviris
import pandas as pd
import pysptools.eea as eea
import pysptools.noise as noise
import pandas as pd
import os

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,classification_report
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier

In [ ]:
img = aviris.open("spectral file")

In [ ]:
img

# PURE PIXEL EXTRACTION

In [ ]:
img_data=img.load()

np.set_printoptions(threshold=np.inf, linewidth=140)

img_2d=img_data.reshape((img_data.shape[0]*img_data.shape[1]), img_data.shape[2])

In [ ]:
[purepixels,bands]=eea.eea.PPI(img_2d,10,10)

In [ ]:
bands

# WAVELENGTH EXTRACTION OF PPI

In [ ]:
file_path = 'spectral file'

In [ ]:
column_names =['wavelength_center_position','fwhm_equivalent_gaussian','Uncertainty in wavelength center position',
                           'Uncertainty in FWHM for equivalent gaussian',
                           'channel_number']

In [ ]:
df_wav_ppi = pd.read_csv(file_path, delimiter='\t', header=None, names=column_names)

In [ ]:
wavelength=[]
wavelength = df_wav_ppi.wavelength_center_position

In [ ]:
len(wavelength)

In [ ]:
wavelength[:10]

In [ ]:
max_ppi,min_ppi=max(wavelength),min(wavelength)

In [ ]:
print(max_ppi)
print(min_ppi)

In [ ]:
ppi_wavelen=[]
for wave in wavelength:
    we=float(round(wave))
    ppi_wavelen.append(we)
wave_spec= pd.DataFrame({'wavelength':ppi_wavelen})

# PPI DATAFRAME

In [ ]:
bands

In [ ]:
def read_and_process_file(file_path, wave_spec):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.strip():
                if "Name: " in line:
                    name_value = line.split("Name: ")[1].strip()
                elif "Class: " in line:
                    class_value = line.split("Class: ")[1].strip()
                elif len(line.split()) == 2 and all(value.replace('.', '').isdigit() for value in line.split()):

                    data.append([float(value) for value in line.split()])

    df = pd.DataFrame(data, columns=['wavelength', f'reflectance#{class_value}#{name_value}'])
    df['wavelength'] = pd.to_numeric(df['wavelength'], errors='coerce') * 1000
    df[f'reflectance#{class_value}#{name_value}'] = pd.to_numeric(df[f'reflectance#{class_value}#{name_value}'], errors='coerce')
    return df[df['wavelength'].isin(wave_spec['wavelength']) == True]


In [ ]:
def merge_files_into_dataframe(directory, wave_spec):
    dfs = pd.DataFrame({'wavelength': wave_spec['wavelength']})

    for filename in os.listdir(directory):
        if filename.endswith('.spectrum.txt'):
            file_path = os.path.join(directory, filename)
            df = read_and_process_file(file_path, wave_spec)
            dfs = pd.merge(dfs, df, on='wavelength', how='left')

    return dfs

In [ ]:
directory = 'output'
result_dataframe = merge_files_into_dataframe(directory, wave_spec)
print(result_dataframe)

In [ ]:
df_transposed = result_dataframe.transpose().reset_index()
df_transposed.set_axis(df_transposed.iloc[0], axis=1, inplace=True)
df_transposed = df_transposed[1:]
df_transposed[['value', 'class', 'sub - class']] = df_transposed['wavelength'].str.split("#",expand=True)

columns_to_drop = ['value', 'wavelength']
df_transposed = df_transposed.drop(columns=columns_to_drop)

filtered_columns = df_transposed.columns[df_transposed.notnull().all()]
filtered_df = df_transposed[filtered_columns]

col=list(filtered_df.columns)[:-2]

filtered_df['class'].value_counts()

In [ ]:
filtered_df.head()

In [ ]:
row_indices_to_print = list(bands) # list of ppi bands

specific_rows = img_2d[row_indices_to_print,:]

df_pppi = pd.DataFrame(specific_rows.T, columns=row_indices_to_print)

df_pppi.insert(0, 'wavelength', ppi_wavelen) #inserting the purepixel wavelengths

selected_df2 = df_pppi[df_pppi['wavelength'].isin(col)]

selecteddf_t=selected_df2.T
selecteddf_t.set_axis(selecteddf_t.iloc[0], axis=1, inplace=True)
selecteddf_t= selecteddf_t[1:]

selecteddf_t

# ML Model

In [ ]:
X = filtered_df.drop(['class','sub - class'], axis=1)
y = filtered_df['class']

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
#set(filtered_df['class'])
# Use Counter to get the frequency of each element
from collections import Counter
frequency_counter = Counter(filtered_df['class'])

# Print unique values and their frequencies
print("Unique values and their frequencies:")
for value, frequency in frequency_counter.items():
    print(f"{value}: {frequency}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 60.47%


In [ ]:

# Standardize the new features using the same scaler used for training data
X_new = scaler.transform(selecteddf_t)

# Make predictions on the new data
y_pred_new = svm_model.predict(X_new)

# If you used LabelEncoder for the target variable, you might want to inverse_transform the predictions
# to get back the original text labels
y_pred_labels = label_encoder.inverse_transform(y_pred_new)

# Display or use the predictions as needed
print("Predictions on new data:")
print(y_pred_labels)

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=12)  # You can adjust the number of neighbors

# Train the KNN model
knn_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = knn_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')



In [ ]:
# Standardize the new features using the same scaler used for training data
X_new = scaler.transform(selecteddf_t)

# Make predictions on the new data
y_pred_new = knn_model.predict(X_new)

y_pred_labels = label_encoder.inverse_transform(y_pred_new)

# Display or use the predictions as needed
print("Predictions on new data:")
print(y_pred_labels)


In [ ]:
# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier on the training data
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Display a classification report
print('Classification Report:')
print(classification_report(y_test, y_pred))

In [ ]:
X_new = scaler.transform(selecteddf_t)

new_predictions = rf_classifier.predict(X_new)

y_pred_labels = label_encoder.inverse_transform(new_predictions)

# Display the predictions
print('Predictions for the new data:')
print(y_pred_labels)

In [ ]:
xgb_classifier = XGBClassifier(objective='multi:softmax', num_class=len(set(y)), random_state=42)

# Train the classifier on the training data
xgb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_classifier.predict(X_test)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')



In [ ]:
# Display a classification report
print('Classification Report:')
print(classification_report(y_test, y_pred))




In [ ]:
X_new = scaler.transform(selecteddf_t)

xgb_new_predictions = xgb_classifier.predict(X_new)

y_pred_labels = label_encoder.inverse_transform(xgb_new_predictions)

# Display the predictions
print('XGBoost Predictions for the new data:')
print(y_pred_labels)